In [8]:
import numpy as np
from tifffile import *
import xml.etree.ElementTree as ET
# sscore on windows

In [9]:
#Input all data from file
def tiff_in(inFile):
	with TiffFile(inFile) as tif:
		tif_tags = {}
		for tag in tif.pages[0].tags.values():
			name, value = tag.name, tag.value
			tif_tags[name] = value
			#images = tif.asarray()
		pages = [tif.asarray(),tif_tags]
		shapes=[]
		for series in tif.series:
			shapes.append(series.shape)
	return pages,shapes

def channelNames(inFile):
    markerDict = {}
    iCnt = 0
    with TiffFile(inFile) as tif:
        for page in tif.series[0].pages:
            markerDict[iCnt] = ET.fromstring(page.description).find('Name').text
            iCnt += 1
    return markerDict

In [11]:
# turn off warnings
import warnings
warnings.filterwarnings("ignore")

In [12]:
import cupy as cp
from scipy.stats import spearmanr
from skimage.transform import pyramid_reduce, rescale
import cupy as cp
import numpy as np

def per_pixel_correlation(image, corrDimensions=1000):
    cp.get_default_memory_pool().free_all_blocks()
    c,n,m  = image.shape
    # Resize the image in a more memory efficient manner
    image_gpu = cp.asarray(image)
    downsampled_image = cp.resize(image_gpu, (c, corrDimensions, corrDimensions))
    print(downsampled_image.shape)
    # make 2d from 3d
    flat_images = cp.reshape(downsampled_image, (c, -1))
    #compute zscore
    flat_images = (flat_images - flat_images.mean(axis=1, keepdims=True)) / flat_images.std(axis=1, keepdims=True)
    # Compute per pixel correlation across all channels
    # Initialize an array to store the Spearman correlation for each channel pair
    correlations = cp.zeros((c, c), dtype=cp.float32)

    for i in range(c):
        print(i)
        for j in range(i, c):
            if i == j:
                # Correlation of a channel with itself is 1
                correlations[i, j] = 1
                continue
            # Compute Spearman correlation for each channel pair
            # change this to pearsonr for Pearson's correlation
            correlation, _ = spearmanr(flat_images[i].get(), flat_images[j].get())
            correlations[i, j] = correlation
            correlations[j, i] = correlation  # Since the correlation matrix is symmetric
            

    return correlations    


In [17]:
sample_files = {
    '3026': '11092021_BaharehNP_3026CtlCorte.qptiff',
    '3628': '12252021_Bahareh-3628-Cntr.qptiff',
    '3280': '12032021-Bahareh_3280_Alz.qptiff',
    '3196': '10122021_BaharehNP_ADCortex_EDTA.qptiff',
    '2997': '10202021_BaharehNP_2997-ADCortex_EDTA.qptiff',
    '3155': '11162021_Bahareh-3155_EDTA.qptiff',
    '1796': '10122021_BaharehNP_1-CtlCortex1796_EDTA_MKT6.qptiff',
    '1873': '10222021_BaharehNP_1873_CtrlCortex.qptiff'
}


In [18]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

for key, file_path in sample_files.items():
    # if file exists, skip
    if os.path.exists(f'corr/{key}_corr.csv'):
        print(f'corr/{key}_corr.csv exists')
        continue
    # read qptiff
    print(key, file_path)
    #2997: IBA 11 70; tmem:50 130
    inFile = file_path
    (all_pages,shapes)=tiff_in(file_path)
    marker_dict = channelNames(file_path)
    outMat = per_pixel_correlation(all_pages[0], 1)
    outDF = pd.DataFrame(outMat, index=marker_dict.values(), columns=marker_dict.values(), dtype='float')
    outDF.to_csv(f'corr/{key}_corr.csv')

import glob
for outF in glob.glob('corr/*.csv'):
    outDF = pd.read_csv(outF, index_col=0)
    key = outF.split('/')[-1].split('_')[0]
    sns.clustermap(outDF, cmap='vlag', figsize=(15,15),
                dendrogram_ratio=0.05)
    plt.savefig(f'corr/{key}_corrPlot.png', dpi=300)


3026 11092021_BaharehNP_3026CtlCorte.qptiff
